In [1]:
#imports used:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter

#not yet used, but imported:
from urllib.request import urlopen
import mechanicalsoup
import time
from requests.auth import HTTPBasicAuth


In [2]:
#my specific credentials for login

login = "amy.jancewicz@gmail.com" 
password = "ngu7RVP@daf2rqw5cnw" 
login_url = "https://wandrer.earth/signin" 
dashboard_url = "https://wandrer.earth/athletes/77230/metro-vancouver-regional-district-british-columbia-canada#7.44/49.287/-123.061"

In [3]:
#create a specific session
with requests.session() as s: 
    
    #use GET command to find the authenticity token within the login url
    req = s.get(login_url).text 
    html = BeautifulSoup(req,"html.parser") 
    token = html.find("input", {"name": "authenticity_token"}).attrs["value"] 

    #use my specific authenticity token plus credentials to access my account
    payload = { 
        "authenticity_token": token, 
        'athlete[email]': login, 
        'athlete[password]': password, 
    }
    
    #response following posting my data to the location above
    res = s.post(login_url, data=payload) 

#report outcome of login activity
if res.ok or res.status_code == 302:
    redirected_url = res.url
    print("Login successful", res.status_code, redirected_url)
else:
    print("Login failed")

Login successful 200 https://wandrer.earth/dashboard


In [4]:
#retrieve the contents of the dashboard site
r = s.get(dashboard_url) 
soup = BeautifulSoup (r.content, "html.parser")
print(dashboard_url, soup)

https://wandrer.earth/athletes/77230/metro-vancouver-regional-district-british-columbia-canada#7.44/49.287/-123.061 <!DOCTYPE html>

<html lang="en">
<head>
<title>Wandrer - Amy Jancewicz (Metro Vancouver Regional District)</title>
<script crossorigin="anonymous" integrity="sha384-wcrNB9xu3AFa0lJwycNeQBrNOYLOZoE4TQotc9kSM8HTuTIB2WEucsI+ZSzuZQCg" src="https://unpkg.com/mapbox-gl@1.13.0/dist/mapbox-gl.js"></script>
<script crossorigin="anonymous" integrity="sha256-hfmVcDB4v86G5tVgzUJ60031uvFWF+38pqVjkCxTkNs=" src="https://cdnjs.cloudflare.com/ajax/libs/intro.js/2.7.0/intro.min.js"></script>
<script crossorigin="anonymous" integrity="sha256-PquXqTvvZ0ST37z1IS11qpxcI5KBF+bJLGqBBY+loW0=" src="https://cdnjs.cloudflare.com/ajax/libs/vanilla-lazyload/8.7.1/lazyload.min.js"></script>
<script crossorigin="anonymous" integrity="sha384-UsmkNeZeX/gA3VvLoHVqR9S5REtaxYj3Wnxuj66ksCAeeLfZxfbmUjnrare7zAIY" src="https://unpkg.com/pbf@3.0.5/dist/pbf.js"></script>
<script crossorigin="anonymous" integrity=

* All code above is used to locate, open, and read the wandrer.earth data. This must be executed properly before moving forward.
* `soup` is now a variable containing the html content of the site `dashboard_url`

In [5]:
#make lists to add the region and progress data
region_gb_list = []
region_name_list = []
region_gps_list = []

region_completed_list = []
region_progress_list = []

#Find all elements with the class "achievement_row"
#In addition to locations, this will also check date achievements, but I can drop those in the df later
achievement_rows = soup.find_all(class_="achievement_row")

#set counters to check NaNs
no_name = 0
no_gb = 0
no_gps = 0

#iterate through each achievement row
for achievement_row in achievement_rows:
    #extract location text
    #if section has name, use that, then check missing name, otherwise 'N/A'
    
    present_name_element = achievement_row.select_one('.achievement_name')
    missing_name_element = achievement_row.select_one('.missing_achievement_name')

    if present_name_element:
        achievement_name = present_name_element.text
        
    elif missing_name_element:
        achievement_name = missing_name_element.text
    else:
        achievement_name = 'N/A'
        no_name = no_name + 1
    
    region_name_list.append(achievement_name)
        
    # Extract "gb ID" value
    data_gb_element = achievement_row.select_one('.geom_toggle')
    data_gb_value = data_gb_element['data-gb'] if data_gb_element else "N/A"
    if data_gb_value == "N/A":
        no_gb = no_gb + 1
    region_gb_list.append(data_gb_value)

    # Extract coordinates from the data-diagonal attribute
    data_diagonal_element = achievement_row.select_one('.geom_toggle')
    coordinates = eval(data_diagonal_element['data-diagonal'])['coordinates'] if data_diagonal_element else "N/A"
    if coordinates == "N/A":
        no_gps = no_gps + 1
    region_gps_list.append(coordinates)

    #print("Location:", achievement_name)
    #print("Data-gb Value:", data_gb_value)
    #print("Coordinates:", coordinates)
    #print("------------------------")

#show the number of missing values vs. found values
print(f"GB IDs \n missing: {no_gb}, present: {len(region_gb_list)-no_gb}")
print(f"Names: \n missing: {no_name}, present: {len(region_name_list)-no_name}")
print(f"Coordinates \n missing: {no_gps}, present: {len(region_gps_list)-no_gps}")

GB IDs 
 missing: 12, present: 162
Names: 
 missing: 2, present: 172
Coordinates 
 missing: 12, present: 162


There are 172 valid Name calls, but only 162 valid Regions. The name calls are likely non-location achievements, which will be dropped in the subsequent df.

Next, I'll scrape sub-sites for each GB value to find the total distance available by plugging the GB value into a url and retrieving the result.

In [36]:
for i in region_gb_list:
    gb_url = ('https://wandrer.earth/geometry_badges/'+i)
    r = s.get(gb_url) 
    sub_soup = BeautifulSoup(r.content, "html.parser")
    total_span = sub_soup.find('span', string=lambda s: 'km total length' in s)
    text = lambda f: f if f else 'NA'

    print(text(total_span))

<span>
      61.4 km total length. Worth 76.4 total points.
    </span>
<span>
      128.5 km total length. Worth 159.7 total points.
    </span>
<span>
      25.5 km total length. Worth 31.7 total points.
    </span>
NA
<span>
      58.5 km total length. Worth 72.6 total points.
    </span>
<span>
      22.2 km total length. Worth 27.6 total points.
    </span>
<span>
      30.3 km total length. Worth 37.7 total points.
    </span>
<span>
      81.4 km total length. Worth 101.2 total points.
    </span>
<span>
      60.6 km total length. Worth 75.3 total points.
    </span>
<span>
      228.8 km total length. Worth 284.4 total points.
    </span>
<span>
      29.9 km total length. Worth 37.2 total points.
    </span>
<span>
      39.7 km total length. Worth 49.4 total points.
    </span>
<span>
      84.8 km total length. Worth 105.3 total points.
    </span>
<span>
      106.6 km total length. Worth 132.4 total points.
    </span>
<span>
      25.3 km total length. Worth 31.5 total p

<span>
      713.6 km total length. Worth 886.8 total points.
    </span>
<span>
      43.8 km total length. Worth 54.4 total points.
    </span>
<span>
      26.0 km total length. Worth 32.3 total points.
    </span>
<span>
      90.7 km total length. Worth 112.7 total points.
    </span>
<span>
      90.4 km total length. Worth 112.3 total points.
    </span>
<span>
      56.1 km total length. Worth 69.7 total points.
    </span>
<span>
      20.4 km total length. Worth 25.4 total points.
    </span>
<span>
      32.4 km total length. Worth 40.3 total points.
    </span>
<span>
      16.9 km total length. Worth 21.0 total points.
    </span>
<span>
      92.2 km total length. Worth 114.6 total points.
    </span>
<span>
      2676.1 km total length. Worth 3325.7 total points.
    </span>
<span>
      36.4 km total length. Worth 45.3 total points.
    </span>
<span>
      1676.2 km total length. Worth 2083.1 total points.
    </span>
<span>
      66.3 km total length. Worth 82.3 total

TypeError: argument of type 'NoneType' is not iterable

In [39]:
#prepare lists for data on distances:
#(by including the code in this block, it will prevent overloading redundant data in these lists if the code is re-run)
region_foot_distance_list = []
region_bike_distance_list = []
region_total_distance_list = []

#look for the summary pages of each of the regions represented by the gb codes, by scraping their individual URLs.
for i in region_gb_list:
    gb_url = ('https://wandrer.earth/geometry_badges/'+i)
    r = s.get(gb_url) 
    sub_soup = BeautifulSoup(r.content, "html.parser")

#retrieve the total distance on bike, foot, and total.
#some gb areas do not have breakdown by bike and foot, assume the total is equal to the distance on bike or foot.

    #foot distance
    foot_span = sub_soup.find('span', string=lambda s: 'foot' in s)

    #bike distance
    bike_span = sub_soup.find('span', string=lambda s: 'bike' in s)
    
    #total distance
        #this is an interesting case, because when bike and run distances are not presented, the string is different.
        #use an if statement to look for cases with all 3 listed, and else find the single total value.
    if sub_soup.find('span', string=lambda s: 'total length' in s):
        total_span = sub_soup.find('span', string=lambda s: 'total length' in s)
    else: total_span = sub_soup.find('span', string=lambda s: 'km. Worth' in s)

    #convert foot, bike, or total found in html to TEXT, strip excess characters
    text = lambda f: f.get_text().strip().replace(',', '') if f else 'NA'
    
    #Append the distances to new lists
    region_foot_distance_list.append(text(foot_span))
    region_bike_distance_list.append(text(bike_span))
    region_total_distance_list.append(text(total_span))

    print(f'{gb_url} \n    {text(foot_span)},\n    {text(bike_span)},\n    {text(total_span)}')

https://wandrer.earth/geometry_badges/324902 
    59.7 km on foot. Worth 74.2 total points.,
    58.1 km on bike. Worth 72.3 total points.,
    61.4 km total length. Worth 76.4 total points.
https://wandrer.earth/geometry_badges/143055 
    128.5 km on foot. Worth 159.7 total points.,
    107.6 km on bike. Worth 133.8 total points.,
    128.5 km total length. Worth 159.7 total points.
https://wandrer.earth/geometry_badges/341677 
    25.4 km on foot. Worth 31.6 total points.,
    24.3 km on bike. Worth 30.2 total points.,
    25.5 km total length. Worth 31.7 total points.
https://wandrer.earth/geometry_badges/343778 
    NA,
    NA,
    0.5 km. Worth 0.6 total points.
https://wandrer.earth/geometry_badges/324852 
    58.4 km on foot. Worth 72.6 total points.,
    55.3 km on bike. Worth 68.7 total points.,
    58.5 km total length. Worth 72.6 total points.
https://wandrer.earth/geometry_badges/350749 
    21.9 km on foot. Worth 27.2 total points.,
    20.6 km on bike. Worth 25.6 total p

https://wandrer.earth/geometry_badges/350178 
    56.3 km on foot. Worth 70.0 total points.,
    53.0 km on bike. Worth 65.8 total points.,
    56.3 km total length. Worth 70.0 total points.
https://wandrer.earth/geometry_badges/324869 
    54.8 km on foot. Worth 68.1 total points.,
    43.9 km on bike. Worth 54.6 total points.,
    55.0 km total length. Worth 68.4 total points.
https://wandrer.earth/geometry_badges/324887 
    21.5 km on foot. Worth 26.7 total points.,
    22.3 km on bike. Worth 27.8 total points.,
    22.6 km total length. Worth 28.1 total points.
https://wandrer.earth/geometry_badges/324870 
    76.4 km on foot. Worth 95.0 total points.,
    75.6 km on bike. Worth 94.0 total points.,
    77.2 km total length. Worth 95.9 total points.
https://wandrer.earth/geometry_badges/350607 
    17.5 km on foot. Worth 21.8 total points.,
    16.7 km on bike. Worth 20.8 total points.,
    17.7 km total length. Worth 22.0 total points.
https://wandrer.earth/geometry_badges/324824 

https://wandrer.earth/geometry_badges/324860 
    NA,
    NA,
    3.4 km. Worth 4.3 total points.
https://wandrer.earth/geometry_badges/142986 
    112.2 km on foot. Worth 139.4 total points.,
    106.7 km on bike. Worth 132.6 total points.,
    112.2 km total length. Worth 139.4 total points.
https://wandrer.earth/geometry_badges/143033 
    244.2 km on foot. Worth 303.5 total points.,
    230.6 km on bike. Worth 286.6 total points.,
    250.7 km total length. Worth 311.6 total points.
https://wandrer.earth/geometry_badges/324894 
    522.5 km on foot. Worth 649.3 total points.,
    489.8 km on bike. Worth 608.8 total points.,
    525.3 km total length. Worth 652.8 total points.
https://wandrer.earth/geometry_badges/324889 
    286.2 km on foot. Worth 355.7 total points.,
    273.4 km on bike. Worth 339.8 total points.,
    295.7 km total length. Worth 367.5 total points.
https://wandrer.earth/geometry_badges/143043 
    192.8 km on foot. Worth 239.6 total points.,
    181.2 km on bik

https://wandrer.earth/geometry_badges/357617 
    1636.8 km on foot. Worth 2034.1 total points.,
    1625.7 km on bike. Worth 2020.3 total points.,
    1676.2 km total length. Worth 2083.1 total points.
https://wandrer.earth/geometry_badges/324861 
    66.0 km on foot. Worth 82.0 total points.,
    60.1 km on bike. Worth 74.7 total points.,
    66.3 km total length. Worth 82.3 total points.
https://wandrer.earth/geometry_badges/143051 
    1225.3 km on foot. Worth 1522.7 total points.,
    1161.0 km on bike. Worth 1442.8 total points.,
    1240.2 km total length. Worth 1541.3 total points.
https://wandrer.earth/geometry_badges/324888 
    133.5 km on foot. Worth 165.9 total points.,
    123.9 km on bike. Worth 154.0 total points.,
    133.9 km total length. Worth 166.5 total points.
https://wandrer.earth/geometry_badges/331628 
    36.0 km on foot. Worth 44.8 total points.,
    33.7 km on bike. Worth 41.9 total points.,
    36.1 km total length. Worth 44.8 total points.
https://wandrer

TypeError: argument of type 'NoneType' is not iterable

In [40]:
for x in [region_gb_list, region_name_list, region_gps_list, region_foot_distance_list, region_bike_distance_list, region_total_distance_list]:
    print(len(x))

174
174
174
162
162
162


In [41]:
data = {'GB code': region_gb_list, 'Locations': region_name_list, 'Coordinates': region_gps_list}
df = pd.DataFrame(data)
df.sample(20)

,GB code,Locations,Coordinates
166,N/A,"March, 2023",N/A
50,355397,First Shaughnessy Heritage Conservation Area,"[[-123.15269, 49.24925], [-123.12794, 49.25795]]"
142,324859,University of British Columbia,"[[-123.26221, 49.24313], [-123.22736, 49.27312]]"
19,343526,Burnaby Mountain,"[[-122.94733, 49.26538], [-122.89359, 49.29239]]"
103,143083,Port Coquitlam,"[[-122.80737, 49.21347], [-122.69847, 49.28927]]"
140,143024,Tsleil Waututh Nation,"[[-122.99136, 49.30321], [-122.88286, 49.47685]]"
18,331629,Burnaby Heights,"[[-123.0234, 49.28101], [-123.00293, 49.29348]]"
109,143073,Richmond,"[[-123.30997, 49.08604], [-122.95721, 49.26194]]"
144,324856,Victoria-Fraserview,"[[-123.07754, 49.20535], [-123.04882, 49.2329]]"
146,325170,West Cambie,"[[-123.12475, 49.17014], [-123.10249, 49.19198]]"


In [42]:
#drop the time achievement rows
df.drop(df[df['GB code'] == 'N/A'].index, inplace=True)
df.shape

(162, 3)

In [57]:
df['Foot'] = region_foot_distance_list
df['Bike'] = region_bike_distance_list
df['Total distance'] = region_total_distance_list
df.shape, df.dtypes

((162, 6),
 GB code           object
 Locations         object
 Coordinates       object
 Foot              object
 Bike              object
 Total distance    object
 dtype: object)

In [44]:
df.head()

,GB code,Locations,Coordinates,Foot,Bike,Total distance
0,324902,Aldergrove,"[[-122.50378, 49.04568], [-122.45964, 49.0677]]",59.7 km on foot. Worth 74.2 total points.,58.1 km on bike. Worth 72.3 total points.,61.4 km total length. Worth 76.4 total points.
1,143055,Anmore,"[[-122.89249, 49.29937], [-122.8225, 49.38255]]",128.5 km on foot. Worth 159.7 total points.,107.6 km on bike. Worth 133.8 total points.,128.5 km total length. Worth 159.7 total points.
2,341677,Annacis Island,"[[-122.98568, 49.15893], [-122.91786, 49.19395]]",25.4 km on foot. Worth 31.6 total points.,24.3 km on bike. Worth 30.2 total points.,25.5 km total length. Worth 31.7 total points.
3,343778,Apodaca Provincial Park,"[[-123.34106, 49.35268], [-123.3355, 49.35582]]",NA,NA,0.5 km. Worth 0.6 total points.
4,324852,Arbutus-Ridge,"[[-123.17826, 49.23459], [-123.14696, 49.25781]]",58.4 km on foot. Worth 72.6 total points.,55.3 km on bike. Worth 68.7 total points.,58.5 km total length. Worth 72.6 total points.


The dataframe has been successfully created, the desired columns are present, and the information appears correct based on manual inspection of my account info online.

The data is all object type. The following updates are needed:

Column | Change needed
-|-
<b>GB code</b> | change to `int` dtype
<b>Locations</b> | none
<b>Coordinates</b> | 1. change to series dtype, 2 calculate midpoint, 3. change to `float` dtype
<b>Foot</b></b> | split to new columns: distance, points. Trim text and change to `float` dtype
<b>Bike</b> | split to new columns: distance, points. Trim text and change to `float` dtype
<b>Total distance</b> |split to new columns: distance, points. Trim text and change to `float` dtype


In [99]:
#update the GB code to float
df['GB code'] = df['GB code'].astype('float64')
df.dtypes

GB code           float64
Locations          object
Coordinates        object
Foot               object
Bike               object
Total distance     object
dtype: object

In [109]:
df.shape

(162, 6)

In [116]:

dfcoords = pd.DataFrame(df['Coordinates'].tolist(), index=df.index)
dfcoords

,0,1
0,"[-122.50378, 49.04568]","[-122.45964, 49.0677]"
1,"[-122.89249, 49.29937]","[-122.8225, 49.38255]"
2,"[-122.98568, 49.15893]","[-122.91786, 49.19395]"
3,"[-123.34106, 49.35268]","[-123.3355, 49.35582]"
4,"[-123.17826, 49.23459]","[-123.14696, 49.25781]"
...,...,...
157,"[-122.67989, 49.1115]","[-122.64702, 49.12627]"
158,"[-122.98898, 49.21505]","[-122.97194, 49.23095]"
159,"[-123.1487, 49.31265]","[-123.12017, 49.32885]"
160,"[-122.80709, 49.22904]","[-122.78659, 49.2521]"


In [94]:
import json
# Define a function to convert the string to a list of coordinates
def parse_coordinates(coord_str):
    coords = json.loads(coord_str)  # Using eval to convert the string to a list
    return pd.Series(coords)

# Apply the function to the 'Coordinates' column and expand into separate columns
df[['Longitude1', 'Latitude1', 'Longitude2', 'Latitude2']] = df['Coordinates'].apply(parse_coordinates)

# Drop the original 'Coordinates' column if needed
#df = df.drop('Coordinates', axis=1)

# Display the resulting DataFrame
print(df)

TypeError: the JSON object must be str, bytes or bytearray, not list

In [ ]:
inprogress_df = df.copy()
inprogress_df['Progress'] = inprogress_df['Progress'].astype(float)
inprogress_df['Region'] = inprogress_df['Region'].astype(str)
inprogress_df = inprogress_df[~inprogress_df['Region'].str.contains('2023')]
inprogress_df.sort_values(by='Progress', inplace=True)
inprogress_df.shape

Next, 
1. find the total distance of the given `Region`
2. IF 
 * distance >75%, find <u>the distance required to complete (>99%) region</u>
 * distance >50%, find <u>the distance required to move to >75%</u>
 * distance >25%, find <u>the distance required to move to >50%</u>
 * distance <25%, find <u>the distance required to move to >25%</u>
3. find the distance of regions that haven't been started required to move to 25%
4. sort by shortest distance to completion for the region

import re

for region in regions:
    
    # Extract text content from the div element
    region_text = region.get_text(separator=' ', strip=True)

    # Use regular expression to find the distance on foot
    match = re.search(r'(\d+\.\d+)\s*km on foot', region_text)

    if match:
        distance_value = match.group(1)
        print(distance_value)



In [ ]:
#identify the region distance element from the popup info on each region
region_code_list = []
region_code = r.find('a', {'class': 'fa fa-info-circle info-circle', 'href': '#', 'data-bs-toggle': 'modal'})

#look for progress
if region_progress_element:
    progress_bar = region_progress_element.find('div', class_='progress-bar')
    region_progress = progress_bar['aria-valuenow']
    region_progress_values.append(region_progress)
    #print(f"Region: {region_name}     Progress: {region_progress}% \n")
else: region_progress_values.append(0)


In [ ]:
#find number code for regions and return ID
if region_distance_element:
    region_distance = region_distance_element.text.strip()

    #add region name to list
    region_distances.append(region_distance)
else: print('none found')

In [ ]:
#make list to add the total region distance data
region_distances = []

for r in regions:
    distance_span = r.find('span', string=lambda s: 'km on foot' in s)
    
    if distance_span:
        # Extract the distance value
        distance_text = distance_span.get_text(strip=True)
        distance_value = distance_text.split()[0]  # Extract the numeric value
        print(distance_value)
    else:
        print("Distance on foot not found.")
    
    #find regions and return distance
    #if distance_element:
        #region_distance = distance_element.text.strip()
        
        #add region name to list
        #region_distances.append(region_distance)

In [ ]:
inprogress_df['Level_Up'] = 0.0

for index, row in inprogress_df.iterrows():
    progress = row['Progress']
    if progress >= 99.0:
        0
    elif progress >= 90.0:
        inprogress_df.at[index, 'Level_Up'] = 99.0 - progress
    elif progress >= 75.0:
        inprogress_df.at[index, 'Level_Up'] = 90.0 - progress
    elif progress >= 50.0:
        inprogress_df.at[index, 'Level_Up'] = 75.0 - progress
    elif progress >= 25.0:
        inprogress_df.at[index, 'Level_Up'] = 50.0 - progress
    else:
        inprogress_df.at[index, 'Level_Up'] = 25.0 - progress
        
inprogress_df.sort_values(by='Level_Up', inplace=True)
inprogress_df.head(12)

In [ ]:
import pandas as pd

# Assuming 'Progress' column exists in your DataFrame
# Create a DataFrame for demonstration purposes
dataa = {'Progress': [80, 60, 45, 30, 20]}
inprogres_df = pd.DataFrame(dataa)

# Apply the conditions and create the 'Level_Up' column
inprogres_df['Level_Up'] = 0  # Initialize the column with zeros

# Update the values based on conditions
inprogres_df.loc[inprogres_df['Progress'] >= 90, 'Level_Up'] = 100 - inprogres_df['Progress']
inprogres_df.loc[(inprogres_df['Progress'] >= 75) & (inprogres_df['Progress'] < 90), 'Level_Up'] = 90 - inprogress_df['Progress']
inprogres_df.loc[(inprogres_df['Progress'] >= 50) & (inprogres_df['Progress'] < 75), 'Level_Up'] = 75 - inprogress_df['Progress']
inprogres_df.loc[(inprogres_df['Progress'] >= 25) & (inprogres_df['Progress'] < 50), 'Level_Up'] = 50 - inprogress_df['Progress']
inprogres_df.loc[inprogres_df['Progress'] < 25, 'Level_Up'] = 25 - inprogres_df['Progress']

# Print the resulting DataFrame
print(inprogres_df)
